## 导入依赖库

In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import pandas as pd
import jax.numpy as jnp
from jax import jit, vmap, random
import interpax


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## 读取数据

In [2]:
def safe_read_dat(dat_name):
    """
    安全读取数据文件，返回numpy数组。
    """
    path = os.path.join('./data', dat_name)
    try:
        data = np.loadtxt(path)
        return data
    except OSError:
        print(f"Cannot find file {path} in current directory")
        return np.array([])


ALPHA1 = safe_read_dat(r'ALPHA1.dat')
BETA1 = safe_read_dat(r'BETA1.dat')
DH1 = safe_read_dat(r'DH1.dat')
Cx = safe_read_dat(r'CX0120_ALPHA1_BETA1_DH1_201.dat')
Cz = safe_read_dat(r'CZ0120_ALPHA1_BETA1_DH1_301.dat')
Cm = safe_read_dat(r'CM0120_ALPHA1_BETA1_DH1_101.dat')

## numpy版本插值

In [3]:
def trilinear_interp_numpy(grid_x, grid_y, grid_z, values, points):
    """
    三维线性插值函数

    参数:
    - grid_x, grid_y, grid_z: 三个维度的网格坐标（升序排列），形状分别为 (nx,), (ny,), (nz,)
    - values: 网格点的值，形状为 (nx, ny, nz)
    - points: 插值点的坐标，形状为 (num_points, 3)

    返回:
    - 插值后的值，形状为 (num_points,)
    """
    x, y, z = points[:, 0], points[:, 1], points[:, 2]
    # 找到每个插值点所在的索引
    ix = np.searchsorted(grid_x, x) - 1
    iy = np.searchsorted(grid_y, y) - 1
    iz = np.searchsorted(grid_z, z) - 1

    # 确保索引在有效范围内
    ix = np.clip(ix, 0, len(grid_x) - 2)
    iy = np.clip(iy, 0, len(grid_y) - 2)
    iz = np.clip(iz, 0, len(grid_z) - 2)

    # 获取立方体的八个顶点的值
    c000 = values[ix    , iy    , iz    ]
    c100 = values[ix + 1, iy    , iz    ]
    c010 = values[ix    , iy + 1, iz    ]
    c110 = values[ix + 1, iy + 1, iz    ]
    c001 = values[ix    , iy    , iz + 1]
    c101 = values[ix + 1, iy    , iz + 1]
    c011 = values[ix    , iy + 1, iz + 1]
    c111 = values[ix + 1, iy + 1, iz + 1]

    # 计算插值权重
    x0 = grid_x[ix]
    x1 = grid_x[ix + 1]
    y0 = grid_y[iy]
    y1 = grid_y[iy + 1]
    z0 = grid_z[iz]
    z1 = grid_z[iz + 1]

    xd = (x - x0) / (x1 - x0)
    yd = (y - y0) / (y1 - y0)
    zd = (z - z0) / (z1 - z0)

    # 插值
    c00 = c000 * (1 - xd) + c100 * xd
    c01 = c001 * (1 - xd) + c101 * xd
    c10 = c010 * (1 - xd) + c110 * xd
    c11 = c011 * (1 - xd) + c111 * xd

    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd

    c = c0 * (1 - zd) + c1 * zd

    return c

In [4]:
ALPHA1_numpy = np.array(ALPHA1)
BETA1_numpy = np.array(BETA1)
DH1_numpy = np.array(DH1)
CX_numpy = np.array(Cx)
CX_numpy = CX_numpy.reshape((DH1_numpy.shape[0], BETA1_numpy.shape[0], ALPHA1_numpy.shape[0]))

n = 10
alpha, beta, el = 90, 30, 25
np.random.seed(42)
alpha_lst= alpha * np.random.uniform(0, 1, (n, 1))
beta_lst = beta * np.random.uniform(0, 1, (n, 1))
el_lst = el * np.random.uniform(0, 1, (n, 1))
input_points = np.hstack((el_lst, beta_lst, alpha_lst))
trilinear_interp_numpy(DH1_numpy, BETA1_numpy, ALPHA1_numpy, CX_numpy, input_points)

array([ 0.09211591,  0.06790906,  0.07973837,  0.09735869,  0.06644634,
        0.02940378, -0.00740422,  0.03257879,  0.07910723,  0.10494726])

In [5]:
%timeit trilinear_interp_numpy(DH1_numpy, BETA1_numpy, ALPHA1_numpy, CX_numpy, input_points)

62.9 μs ± 1.21 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## torch版本插值

In [6]:
def trilinear_interp_tensor(grid_x, grid_y, grid_z, values, points):
    """
    三维线性插值函数

    参数:
    - grid_x, grid_y, grid_z: 三个维度的网格坐标（升序排列），形状分别为 (nx,), (ny,), (nz,)
    - values: 网格点的值，形状为 (nx, ny, nz)
    - points: 插值点的坐标，形状为 (num_points, 3)

    返回:
    - 插值后的值，形状为 (num_points,)
    """
    x, y, z = points[:, 0].contiguous(), points[:, 1].contiguous(), points[:, 2].contiguous()
    # 找到每个插值点所在的索引
    ix = torch.searchsorted(grid_x, x) - 1
    iy = torch.searchsorted(grid_y, y) - 1
    iz = torch.searchsorted(grid_z, z) - 1

    # 确保索引在有效范围内
    ix = torch.clip(ix, 0, len(grid_x) - 2)
    iy = torch.clip(iy, 0, len(grid_y) - 2)
    iz = torch.clip(iz, 0, len(grid_z) - 2)

    # 获取立方体的八个顶点的值
    c000 = values[ix    , iy    , iz    ]
    c100 = values[ix + 1, iy    , iz    ]
    c010 = values[ix    , iy + 1, iz    ]
    c110 = values[ix + 1, iy + 1, iz    ]
    c001 = values[ix    , iy    , iz + 1]
    c101 = values[ix + 1, iy    , iz + 1]
    c011 = values[ix    , iy + 1, iz + 1]
    c111 = values[ix + 1, iy + 1, iz + 1]

    # 计算插值权重
    x0 = grid_x[ix]
    x1 = grid_x[ix + 1]
    y0 = grid_y[iy]
    y1 = grid_y[iy + 1]
    z0 = grid_z[iz]
    z1 = grid_z[iz + 1]

    xd = (x - x0) / (x1 - x0)
    yd = (y - y0) / (y1 - y0)
    zd = (z - z0) / (z1 - z0)

    # 插值
    c00 = c000 * (1 - xd) + c100 * xd
    c01 = c001 * (1 - xd) + c101 * xd
    c10 = c010 * (1 - xd) + c110 * xd
    c11 = c011 * (1 - xd) + c111 * xd

    c0 = c00 * (1 - yd) + c10 * yd
    c1 = c01 * (1 - yd) + c11 * yd

    c = c0 * (1 - zd) + c1 * zd

    return c

In [7]:
ALPHA1_tensor = torch.tensor(ALPHA1, device=device)
BETA1_tensor = torch.tensor(BETA1, device=device)
DH1_tensor = torch.tensor(DH1, device=device)
CX_tensor = torch.tensor(Cx, device=device)
CX_tensor = CX_tensor.reshape((DH1_tensor.shape[0], BETA1_tensor.shape[0], ALPHA1_tensor.shape[0]))

n = 10
alpha, beta, el = 90, 30, 25
torch.manual_seed(42)
alpha_lst= alpha * torch.rand((n, 1), device=device)
beta_lst = beta * torch.rand((n, 1), device=device)
el_lst = el * torch.rand((n, 1), device=device)
input_points = torch.hstack((el_lst, beta_lst, alpha_lst))
trilinear_interp_tensor(DH1_tensor, BETA1_tensor, ALPHA1_tensor, CX_tensor, input_points)

tensor([ 0.0566,  0.0357,  0.0807, -0.0085,  0.1090,  0.0999,  0.0963,  0.0738,
         0.0362,  0.0652], dtype=torch.float64)

In [8]:
%timeit trilinear_interp_tensor(DH1_tensor, BETA1_tensor, ALPHA1_tensor, CX_tensor, input_points)

270 μs ± 18.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## MLP版本

In [9]:
class MLP(nn.Module):
    def __init__(self, in_dim, out_dim, hidden_list):
        super().__init__()
        layers = []
        lastv = in_dim
        for hidden in hidden_list:
            layers.append(nn.Linear(lastv, hidden))
            layers.append(nn.ReLU())
            lastv = hidden
        layers.append(nn.Linear(lastv, out_dim))
        self.layers = nn.Sequential(*layers)
        self.out_dim = out_dim

    def forward(self, x):
        x = x.to(torch.float32)
        ret = self.layers(x)
        ret = ret.reshape(-1)
        return ret
    

def normalize(X, mean, std):
    return (X - mean) / std


def unnormalize(X, mean, std):
    return X * std + mean

In [10]:
data = pd.read_csv('./model/mean_std.csv')
Cx_model = MLP(3, 1, [20, 10]).to(device=device)
Cx_model.load_state_dict(torch.load('./model/Cx.pth', map_location=device))
Cz_model = MLP(3, 1, [20, 10]).to(device=device)
Cz_model.load_state_dict(torch.load('./model/Cz.pth', map_location=device))
Cm_model = MLP(3, 1, [20, 10]).to(device=device)
Cm_model.load_state_dict(torch.load('./model/Cm.pth', map_location=device))

C:\Users\xcy\AppData\Local\Temp\ipykernel_24212\4152871600.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  Cx_model.load_state_dict(torch.load('./model/Cx.pth', map_loca

<All keys matched successfully>

In [11]:
@torch.no_grad()
def _Cx(alpha, beta, dele):
    name = data['name']
    index = list(name).index('Cx')
    alpha_mean = data['alpha_mean'][index]
    alpha_std = data['alpha_std'][index]
    alpha = normalize(alpha, alpha_mean, alpha_std)
    beta_mean = data['beta_mean'][index]
    beta_std = data['beta_std'][index]
    beta = normalize(beta, beta_mean, beta_std)
    el_mean = data['el_mean'][index]
    el_std = data['el_std'][index]
    dele = normalize(dele, el_mean, el_std)
    input = torch.hstack((alpha.reshape(-1, 1), beta.reshape(-1, 1)))
    input = torch.hstack((input, dele.reshape(-1, 1)))
    mean = data['mean'][index]
    std = data['std'][index]
    return unnormalize(Cx_model.forward(input), mean, std)

@torch.no_grad()
def _Cz(alpha, beta, dele):
    name = data['name']
    index = list(name).index('Cz')
    alpha_mean = data['alpha_mean'][index]
    alpha_std = data['alpha_std'][index]
    alpha = normalize(alpha, alpha_mean, alpha_std)
    beta_mean = data['beta_mean'][index]
    beta_std = data['beta_std'][index]
    beta = normalize(beta, beta_mean, beta_std)
    el_mean = data['el_mean'][index]
    el_std = data['el_std'][index]
    dele = normalize(dele, el_mean, el_std)
    input = torch.hstack((alpha.reshape(-1, 1), beta.reshape(-1, 1)))
    input = torch.hstack((input, dele.reshape(-1, 1)))
    mean = data['mean'][index]
    std = data['std'][index]
    return unnormalize(Cz_model.forward(input), mean, std)

@torch.no_grad()
def _Cm(alpha, beta, dele):
    name = data['name']
    index = list(name).index('Cm')
    alpha_mean = data['alpha_mean'][index]
    alpha_std = data['alpha_std'][index]
    alpha = normalize(alpha, alpha_mean, alpha_std)
    beta_mean = data['beta_mean'][index]
    beta_std = data['beta_std'][index]
    beta = normalize(beta, beta_mean, beta_std)
    el_mean = data['el_mean'][index]
    el_std = data['el_std'][index]
    dele = normalize(dele, el_mean, el_std)
    input = torch.hstack((alpha.reshape(-1, 1), beta.reshape(-1, 1)))
    input = torch.hstack((input, dele.reshape(-1, 1)))
    mean = data['mean'][index]
    std = data['std'][index]
    return unnormalize(Cm_model.forward(input), mean, std)

In [12]:
n = 10
alpha, beta, el = 90, 30, 25
torch.manual_seed(42)
alpha_lst= alpha * torch.rand((n, 1), device=device)
beta_lst = beta * torch.rand((n, 1), device=device)
el_lst = el * torch.rand((n, 1), device=device)
_Cx(alpha_lst, beta_lst, el_lst)

tensor([ 0.0569,  0.0363,  0.0922, -0.0021,  0.1106,  0.0975,  0.0899,  0.0771,
         0.0347,  0.0606])

In [13]:
%timeit _Cx(alpha_lst, beta_lst, el_lst)

138 μs ± 806 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## JAX版本插值

In [14]:
@jit
def trilinear_interp_jax(grid_x, grid_y, grid_z, values, points):
    """
    三维线性插值函数

    参数:
    - grid_x, grid_y, grid_z: 三个维度的网格坐标（升序排列），形状分别为 (nx,), (ny,), (nz,)
    - values: 网格点的值，形状为 (nx, ny, nz)
    - points: 插值点的坐标，形状为 (num_points, 3)

    返回:
    - 插值后的值，形状为 (num_points,)
    """
    def single_interp(point):
        x, y, z = point
        # 找到每个插值点所在的索引
        ix = jnp.searchsorted(grid_x, x) - 1
        iy = jnp.searchsorted(grid_y, y) - 1
        iz = jnp.searchsorted(grid_z, z) - 1

        # 确保索引在有效范围内
        ix = jnp.clip(ix, 0, len(grid_x) - 2)
        iy = jnp.clip(iy, 0, len(grid_y) - 2)
        iz = jnp.clip(iz, 0, len(grid_z) - 2)

        # 获取立方体的八个顶点的值
        c000 = values[ix    , iy    , iz    ]
        c100 = values[ix + 1, iy    , iz    ]
        c010 = values[ix    , iy + 1, iz    ]
        c110 = values[ix + 1, iy + 1, iz    ]
        c001 = values[ix    , iy    , iz + 1]
        c101 = values[ix + 1, iy    , iz + 1]
        c011 = values[ix    , iy + 1, iz + 1]
        c111 = values[ix + 1, iy + 1, iz + 1]

        # 计算插值权重
        x0 = grid_x[ix]
        x1 = grid_x[ix + 1]
        y0 = grid_y[iy]
        y1 = grid_y[iy + 1]
        z0 = grid_z[iz]
        z1 = grid_z[iz + 1]

        xd = (x - x0) / (x1 - x0)
        yd = (y - y0) / (y1 - y0)
        zd = (z - z0) / (z1 - z0)

        # 插值
        c00 = c000 * (1 - xd) + c100 * xd
        c01 = c001 * (1 - xd) + c101 * xd
        c10 = c010 * (1 - xd) + c110 * xd
        c11 = c011 * (1 - xd) + c111 * xd

        c0 = c00 * (1 - yd) + c10 * yd
        c1 = c01 * (1 - yd) + c11 * yd

        c = c0 * (1 - zd) + c1 * zd

        return c

    # 使用 vmap 对每个插值点应用插值函数
    interpolated = vmap(single_interp)(points)
    return interpolated


In [15]:
ALPHA1_jnp = jnp.array(ALPHA1)
BETA1_jnp = jnp.array(BETA1)
DH1_jnp = jnp.array(DH1)
CX_jnp = jnp.array(Cx)
CX_jnp = CX_jnp.reshape((DH1_jnp.shape[0], BETA1_jnp.shape[0], ALPHA1_jnp.shape[0]))

n = 10
alpha, beta, el = 90, 30, 25

key = random.PRNGKey(42)
alpha_key, beta_key, el_key = random.split(key, 3)
alpha_lst = alpha * random.uniform(alpha_key, (n, 1))
beta_lst = beta * random.uniform(beta_key, (n, 1))
el_lst = el * random.uniform(el_key, (n, 1))
input_points = jnp.hstack((el_lst, beta_lst, alpha_lst))

trilinear_interp_jax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

Array([ 0.01771834,  0.09925664,  0.10315857,  0.05268927,  0.06349808,
        0.04405698, -0.07633661,  0.05153302,  0.05967385,  0.04271419],      dtype=float32)

In [16]:
%timeit trilinear_interp_jax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

8.79 μs ± 211 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## interpax版本插值

In [17]:
@jit
def trilinear_interpax(grid_x, grid_y, grid_z, values, points):
    """
    三维线性插值函数

    参数:
    - grid_x, grid_y, grid_z: 三个维度的网格坐标（升序排列），形状分别为 (nx,), (ny,), (nz,)
    - values: 网格点的值，形状为 (nx, ny, nz)
    - points: 插值点的坐标，形状为 (num_points, 3)

    返回:
    - 插值后的值，形状为 (num_points,)
    """
    interplot3d = interpax.Interpolator3D(grid_x, grid_y, grid_z, values)

    def single_interp(point):
        x, y, z = point
        return interplot3d(x, y, z)

    # 使用 vmap 对每个插值点应用插值函数
    interpolated = vmap(single_interp)(points)
    return interpolated

In [18]:
ALPHA1_jnp = jnp.array(ALPHA1)
BETA1_jnp = jnp.array(BETA1)
DH1_jnp = jnp.array(DH1)
CX_jnp = jnp.array(Cx)
CX_jnp = CX_jnp.reshape((DH1_jnp.shape[0], BETA1_jnp.shape[0], ALPHA1_jnp.shape[0]))

n = 10
alpha, beta, el = 90, 30, 25

key = random.PRNGKey(42)
alpha_key, beta_key, el_key = random.split(key, 3)
alpha_lst = alpha * random.uniform(alpha_key, (n, 1))
beta_lst = beta * random.uniform(beta_key, (n, 1))
el_lst = el * random.uniform(el_key, (n, 1))
input_points = jnp.hstack((el_lst, beta_lst, alpha_lst))

trilinear_interpax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

Array([ 0.01740061,  0.09906071,  0.10461072,  0.05284601,  0.0621539 ,
        0.04622418, -0.07554753,  0.05037436,  0.06079885,  0.04157751],      dtype=float32)

In [19]:
%timeit trilinear_interpax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

39.3 μs ± 1.25 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## 测试时间

### numpy版本

In [20]:
ns = [10, 100, 1000, 10000, 100000, 1000000]

for n in ns:
    alpha, beta, el = 90, 30, 25
    alpha_lst= alpha * np.random.uniform(0, 1, (n, 1))
    beta_lst = beta * np.random.uniform(0, 1, (n, 1))
    el_lst = el * np.random.uniform(0, 1, (n, 1))
    input_points = np.hstack((el_lst, beta_lst, alpha_lst))
    %timeit trilinear_interp_numpy(DH1_numpy, BETA1_numpy, ALPHA1_numpy, CX_numpy, input_points)

68.5 μs ± 2.46 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
76.1 μs ± 2.08 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
167 μs ± 1.16 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
1.33 ms ± 4.37 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
21.9 ms ± 438 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
269 ms ± 15.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


### torch版本

In [21]:
ns = [10, 100, 1000, 10000, 100000, 1000000]

for n in ns:
    alpha, beta, el = 90, 30, 25
    alpha_lst= alpha * torch.rand((n, 1), device=device)
    beta_lst = beta * torch.rand((n, 1), device=device)
    el_lst = el * torch.rand((n, 1), device=device)
    input_points = torch.hstack((el_lst, beta_lst, alpha_lst))
    %timeit trilinear_interp_tensor(DH1_tensor, BETA1_tensor, ALPHA1_tensor, CX_tensor, input_points)

288 μs ± 10.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
282 μs ± 2.44 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
467 μs ± 6.43 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
1.25 ms ± 64.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
7.05 ms ± 387 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
55.5 ms ± 2.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


### MLP版本

In [22]:
ns = [10, 100, 1000, 10000, 100000, 1000000]
for n in ns:
    alpha, beta, el = 90, 30, 25
    alpha_lst= alpha * torch.rand((n, 1), device=device)
    beta_lst = beta * torch.rand((n, 1), device=device)
    el_lst = el * torch.rand((n, 1), device=device)
    %timeit _Cx(alpha_lst, beta_lst, el_lst)

150 μs ± 2.57 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
149 μs ± 4.75 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
218 μs ± 3.88 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
552 μs ± 6.19 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
2.72 ms ± 178 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)
26.6 ms ± 134 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


### jax版本

In [24]:
ns = [10, 100, 1000, 10000, 100000, 1000000]
# ns = [1000000]
for n in ns:
    alpha, beta, el = 90, 30, 25

    key = random.PRNGKey(42)
    alpha_key, beta_key, el_key = random.split(key, 3)
    alpha_lst = alpha * random.uniform(alpha_key, (n, 1))
    beta_lst = beta * random.uniform(beta_key, (n, 1))
    el_lst = el * random.uniform(el_key, (n, 1))
    input_points = jnp.hstack((el_lst, beta_lst, alpha_lst))

    %timeit trilinear_interp_jax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

9.45 μs ± 270 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
9.82 μs ± 460 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
48.8 μs ± 2.72 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
303 μs ± 18.3 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 10.85 times longer than the fastest. This could mean that an intermediate result is being cached.
22.4 μs ± 26.5 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
33.6 ms ± 1.13 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


### interpax版本

In [25]:
ns = [10, 100, 1000, 10000, 100000, 1000000]
for n in ns:
    alpha, beta, el = 90, 30, 25

    key = random.PRNGKey(42)
    alpha_key, beta_key, el_key = random.split(key, 3)
    alpha_lst = alpha * random.uniform(alpha_key, (n, 1))
    beta_lst = beta * random.uniform(beta_key, (n, 1))
    el_lst = el * random.uniform(el_key, (n, 1))
    input_points = jnp.hstack((el_lst, beta_lst, alpha_lst))

    %timeit trilinear_interpax(DH1_jnp, BETA1_jnp, ALPHA1_jnp, CX_jnp, input_points)

35.7 μs ± 2.23 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
The slowest run took 6.15 times longer than the fastest. This could mean that an intermediate result is being cached.
34.7 μs ± 31.4 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 8.27 times longer than the fastest. This could mean that an intermediate result is being cached.
24.8 μs ± 28.7 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 11.81 times longer than the fastest. This could mean that an intermediate result is being cached.
27.9 μs ± 37.8 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 10.54 times longer than the fastest. This could mean that an intermediate result is being cached.
59 μs ± 62.1 μs per loop (mean ± std. dev. of 7 runs, 1 loop each)
The slowest run took 11.55 times longer than the fastest. This could mean that an intermediate result is being cached.
25.3 μs ± 30.4 μs per loop (mean ± std. dev. of 7 runs, 1 